# **Exploratory Data Analysis using SQL**

## Analyzing Apple App Store Datasets

The project is centered around conducting in-depth exploratory data analysis using SQL to provide valuable insights and actionable recommendations specifically catered to app developers.

check the number of unique apps in both tables

In [1]:
--displays the number of unique value in the column id from the table AppleStore
SELECT 
    COUNT(DISTINCT id) AS UniqueAppsIDs
FROM 
    Projects.dbo.AppleStore

----displays the number of unique value in the column id from the table appleStore_description
SELECT 
    COUNT(DISTINCT id) AS UniqueAppsIDs
FROM 
    Projects.dbo.appleStore_description

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.090

UniqueAppsIDs
7197


UniqueAppsIDs
7197


check for any missing values in key fields

In [2]:
--display the number of missing values on track_name, user_rating, and prime_genre columns on AppleStore table
SELECT 
    COUNT(*) AS MissingValues
FROM   
    Projects.dbo.AppleStore
WHERE 
    track_name IS NULL OR user_rating IS NULL OR prime_genre IS NULL

--display the number of missing values on app_desc column on appleStore_description table
SELECT 
    COUNT(*) AS MissingValues
FROM   
    Projects.dbo.appleStore_description
WHERE 
    app_desc IS NULL 

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.151

MissingValues
0


MissingValues
0


find out the number of apps per genre

In [3]:
--displays the number of apps per genre in descending order
SELECT
    prime_genre,
    COUNT(id) AS NumberOfApps
FROM
    Projects.dbo.AppleStore
GROUP BY 
    prime_genre
ORDER BY 
    NumberOfApps DESC

(23 rows affected)

Total execution time: 00:00:00.012

prime_genre,NumberOfApps
Games,3862
Entertainment,535
Education,453
Photo & Video,349
Utilities,248
Health & Fitness,180
Productivity,178
Social Networking,167
Lifestyle,144
Music,138


get an overview of the apps' rating

In [4]:
SELECT
    MIN(user_rating) AS MinUserRating,
    MAX(user_rating) AS MaxUserRating,
    AVG(user_rating) AS AvgUserRating
FROM 
    Projects.dbo.AppleStore

(1 row affected)

Total execution time: 00:00:00.016

MinUserRating,MaxUserRating,AvgUserRating
0,5,3.526955675976101


Determine whether paid apps have higher ratings than free apps

In [5]:
--get each average rating of paid and free apps
SELECT
    CASE 
        WHEN price > 0 THEN 'Paid'
        ELSE 'Free' 
    END AS App_Type,
    AVG(user_rating) AS AvgRating
FROM 
    Projects.dbo.AppleStore
--repeating same case expression because App_Type doesn't get recognize in group by section
GROUP BY
    CASE 
        WHEN price > 0 THEN 'Paid'
        ELSE 'Free' 
    END

(2 rows affected)

Total execution time: 00:00:00.034

App_Type,AvgRating
Paid,3.720948742438714
Free,3.3767258382642997


check if apps with more supported languages have higher ratings 

In [6]:
-- CTE named LanguageBracketCTE calculates the LanguageBracket using the CASE expression and includes the user_rating column.
WITH LanguageBracketCTE AS (
    SELECT 
        CASE
            WHEN lang_num < 10 THEN '<10 languages'
            WHEN lang_num BETWEEN 10 AND 30 THEN '10 - 30 Languages'
            ELSE '>30 languages'
        END AS LanguageBracket,
        user_rating
    FROM 
        Projects.dbo.AppleStore
)
--The main SELECT statement then performs the aggregation on the LanguageBracket column and calculates the average user rating for each bracket.
SELECT
    LanguageBracket,
    AVG(user_rating) AS AvgRating
FROM 
    LanguageBracketCTE
GROUP BY 
    LanguageBracket
ORDER BY 
    AvgRating DESC;

(3 rows affected)

Total execution time: 00:00:00.046

LanguageBracket,AvgRating
10 - 30 Languages,4.1305120910384066
>30 languages,3.7777777777777777
<10 languages,3.368327402135231


check genres with low ratings

In [7]:
SELECT TOP 10
    prime_genre,
    AVG(user_rating) AS AvgRating
FROM 
    Projects.dbo.AppleStore
GROUP BY
    prime_genre
ORDER BY 
    AvgRating

(10 rows affected)

Total execution time: 00:00:00.025

prime_genre,AvgRating
Catalogs,2.1
Finance,2.4326923076923075
Book,2.4776785714285716
Navigation,2.6847826086956523
Lifestyle,2.8055555555555554
News,2.98
Sports,2.982456140350877
Social Networking,2.9850299401197606
Food & Drink,3.1825396825396823
Entertainment,3.2467289719626167


check if there is a correlation between the length of the app description and the user rating

In [8]:
--the CTE named DescriptionLengthCTE calculates the DescLengthBracket using the CASE expression and includes the id column
WITH DescriptionLengthCTE AS (
    SELECT 
        CASE 
            WHEN LEN(app_desc) < 500 THEN 'Short'
            WHEN LEN(app_desc) BETWEEN 500 AND 1000 THEN 'Medium'
            ELSE 'Long'
        END AS DescLengthBracket,
        id
    FROM 
        Projects.dbo.appleStore_description
)
--The main SELECT statement then performs the aggregation on the DescLengthBracket column and calculates the average user rating for each description length bracket
SELECT
    DescLengthBracket,
    AVG(S.user_rating) AS AvgRating
FROM 
    DescriptionLengthCTE D
JOIN Projects.dbo.AppleStore AS S
ON D.id = S.id
GROUP BY 
    DescLengthBracket
ORDER BY 
    AvgRating

(3 rows affected)

Total execution time: 00:00:00.068

DescLengthBracket,AvgRating
Short,2.533613445378151
Medium,3.232809430255403
Long,3.855946944988041


check the top-rated apps for each genre

In [9]:
SELECT
    prime_genre,
    track_name,
    user_rating
FROM (
        SELECT 
            prime_genre,
            track_name,
            user_rating,
            RANK() OVER (PARTITION BY prime_genre ORDER BY user_rating DESC, rating_count_tot DESC) AS rank 
        FROM 
            Projects.dbo.AppleStore
     ) AS a 
WHERE 
    a.rank = 1

(23 rows affected)

Total execution time: 00:00:00.038

prime_genre,track_name,user_rating
Book,Color Therapy Adult Coloring Book for Adults,5
Business,TurboScan™ Pro - document & receipt scanner: scan multiple pages and photos to PDF,5
Catalogs,CPlus for Craigslist app - mobile classifieds,5
Education,Elevate - Brain Training and Games,5
Entertainment,Bruh-Button,5
Finance,"Credit Karma: Free Credit Scores, Reports & Alerts",5
Food & Drink,Domino's Pizza USA,5
Games,Head Soccer,5
Health & Fitness,Yoga Studio,5
Lifestyle,"ipsy - Makeup, subscription and beauty tips",5


# **Findings**

### 

1. <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">Paid apps tend to receive higher ratings from users.</span>

1. <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">Apps that support between 10 and 30 languages generally receive better ratings.</span>

1. <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">Finance and book genre apps tend to have lower ratings compared to other genres.</span>

1. <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">Apps with more detailed and longer descriptions typically receive better ratings.</span>

1. <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">For a new app, it's important to target an average rating of at least 3.5 to ensure a good user response.</span>

1. <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">The competition is fierce in the games and entertainment genres, so it's important to stand out with unique features.</span>